Hello this is Jensen's branch

In [3]:
!pip install numpy; random; matplotlib; scipy

zsh:1: command not found: random
zsh:1: command not found: matplotlib
zsh:1: command not found: scipy


In [4]:
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [5]:
def gillespie(initial, beta, gamma, mu, max_time, rng):
    S, I, R = initial

    # initialize
    SIR_states = [[S, I, R]]
    timestamps = [0.0]

    # event updates
    events = np.array([
        [ 1,  0,  0],   # Birth
        [-1, +1,  0],   # Infection
        [ 0, -1, +1],   # Recovery
        [-1,  0,  0],   # Death S
        [ 0, -1,  0],   # Death I
        [ 0,  0, -1]    # Death R
    ])

    while timestamps[-1] < max_time and (S+I+R) > 0:
        N = S + I + R

        # propensities (rates)
        event_rates = np.array([
            mu * N,
            beta * S*I/N if N > 0 else 0,
            gamma * I,
            mu * S,
            mu * I,
            mu * R
        ])

        # random numbers for each event
        U = rng.random(len(event_rates))

        # waiting times
        dt_list = np.where(event_rates > 0, -np.log(U) / event_rates, np.inf)

        # next event
        dt_min = np.min(dt_list)
        if dt_min == np.inf:
            break
        event = np.argmin(dt_list)

        # update time
        timestamps.append(timestamps[-1] + dt_min)

        # update state
        dS, dI, dR = events[event]
        S, I, R = max(0, S+dS), max(0, I+dI), max(0, R+dR)

        SIR_states.append([S,I,R])

    return np.array(timestamps), np.array(SIR_states)

In [6]:
def SIR_demography(y, t, N, beta, gamma, mu):
    '''SIR model with demography'''
    
    dSdt = mu - beta*y[0]*y[1] - mu*y[0]
    dIdt = beta*y[0]*y[1] - gamma*y[1] - mu*y[1]
    dRdt = gamma*y[1] - mu*y[2]
    
    return [dSdt, dIdt, dRdt]

In [8]:
# Define population parameters
N = 10000
I0 = 10

# Define model parameters
beta = 1/2
gamma = 1/7
mu = 0.02

# Duration of simulation
days = 100

# Parameters for gillepsie algorithm
initial = [N-I0, I0, 0]
event_rates = [mu, beta, gamma, mu, mu, mu]

# Run gillepsie algorithm
time, values = gillespie(initial, beta, gamma, mu, event_rates, 100)

# Gather values over time for each compartment
susceptible = [event[0] for event in values]
infected = [event[1] for event in values]
recovered = [event[2] for event in values]

# Inital parameters deterministic model
y0 = [(N-I0)/N, I0/N, 0]

# Determine timesteps and resolution
t = np.linspace(0, days, 1001)

# Get deteministic solution
solution = odeint(SIR_demography, y0, t, args=(N, beta, gamma, mu))

# Plot data
fig = plt.figure(figsize=(5,4))

# Plot deteministic model
plt.plot(t, solution[:,0] * 10000,'b--', alpha=0.5)
plt.plot(t, solution[:,1] * 10000,'r--', alpha=0.5)
plt.plot(t, solution[:,2] * 10000,'g--', alpha=0.5)

# Plot gillepsie model
plt.plot(time, susceptible, 'b-', label=r'Susceptible')
plt.plot(time, infected, 'r-', label=r'Infected')
plt.plot(time, recovered, 'g-', label=r'Recovered')

plt.ylabel('Individuals')
plt.xlabel('Time (days)')
plt.legend(loc='best')
plt.show()

TypeError: '<' not supported between instances of 'float' and 'list'